In [79]:
import warnings
warnings.filterwarnings('ignore')

In [80]:
from flowermd.base import Pack, Simulation
from flowermd.library import PPS, BeadSpring
from flowermd.utils import get_target_box_mass_density
import unyt as u

In [81]:
pps_mol = PPS(num_mols=20, lengths=60)

In [82]:
pps_mol.coarse_grain(beads={"A": "c1ccc(S)cc1"})

In [83]:
ff = BeadSpring(
    r_cut=2.5,
    beads={
        "A": dict(epsilon=1, sigma=0.2),
    },
    bonds={
        "A-A": dict(r0=0.64, k=500),
    },
    angles={"A-A-A": dict(t0=2.8, k=50)},
)

In [84]:
ff.hoomd_forces

In [85]:
cg_system = Pack(molecules=pps_mol, density=1, edge=.2, overlap=.2,packing_expand_factor=5)

In [86]:
gsd_path='diameterT6D.gsd'
cg_sim = Simulation(
    initial_state=cg_system.hoomd_snapshot,
    forcefield=ff.hoomd_forces,
    gsd_write_freq=int(2e5/10),
    gsd_file_name=gsd_path,
    reference_values={"length": 1*u.nm, "mass": 1*u.Unit("amu"), "energy":1*u.Unit("kJ/mol")}
)

Initializing simulation state from a gsd.hoomd.Frame.


In [57]:
target_box = get_target_box_mass_density(density=0.5*u.Unit("g/cm**3"), mass=cg_system.mass.to("g"))
print(target_box)

[7.55471261e-07 7.55471261e-07 7.55471261e-07] cm


In [58]:
cg_sim.run_update_volume(n_steps=4e5, kT=3.0, tau_kt=1.0, final_box_lengths=target_box, period=100)

Step 10500 of 300000; TPS: 2456.22; ETA: 2.0 minutes
Step 21000 of 300000; TPS: 2626.52; ETA: 1.8 minutes
Step 31500 of 300000; TPS: 2676.03; ETA: 1.7 minutes
Step 42000 of 300000; TPS: 2684.44; ETA: 1.6 minutes
Step 52500 of 300000; TPS: 2672.3; ETA: 1.5 minutes
Step 63000 of 300000; TPS: 2651.96; ETA: 1.5 minutes
Step 73500 of 300000; TPS: 2624.23; ETA: 1.4 minutes
Step 84000 of 300000; TPS: 2600.63; ETA: 1.4 minutes
Step 94500 of 300000; TPS: 2575.63; ETA: 1.3 minutes
Step 105000 of 300000; TPS: 2547.46; ETA: 1.3 minutes
Step 115500 of 300000; TPS: 2505.61; ETA: 1.2 minutes
Step 126000 of 300000; TPS: 2456.39; ETA: 1.2 minutes
Step 136500 of 300000; TPS: 2412.26; ETA: 1.1 minutes
Step 147000 of 300000; TPS: 2363.3; ETA: 1.1 minutes
Step 157500 of 300000; TPS: 2313.92; ETA: 1.0 minutes
Step 168000 of 300000; TPS: 2254.65; ETA: 1.0 minutes
Step 178500 of 300000; TPS: 2194.99; ETA: 0.9 minutes
Step 189000 of 300000; TPS: 2145.14; ETA: 0.9 minutes
Step 199500 of 300000; TPS: 2090.18; ET

In [88]:
cg_sim.flush_writers()

In [87]:
cg_sim.run_NVT(n_steps=4e5, kT=3.0, tau_kt=1.0)

Step 10500 of 400000; TPS: 2685.95; ETA: 2.4 minutes
Step 21000 of 400000; TPS: 2786.66; ETA: 2.3 minutes
Step 31500 of 400000; TPS: 2826.38; ETA: 2.2 minutes
Step 42000 of 400000; TPS: 2838.9; ETA: 2.1 minutes
Step 52500 of 400000; TPS: 2856.92; ETA: 2.0 minutes
Step 63000 of 400000; TPS: 2867.29; ETA: 2.0 minutes
Step 73500 of 400000; TPS: 2878.96; ETA: 1.9 minutes
Step 84000 of 400000; TPS: 2885.31; ETA: 1.8 minutes
Step 94500 of 400000; TPS: 2890.3; ETA: 1.8 minutes
Step 105000 of 400000; TPS: 2895.63; ETA: 1.7 minutes
Step 115500 of 400000; TPS: 2898.83; ETA: 1.6 minutes
Step 126000 of 400000; TPS: 2902.56; ETA: 1.6 minutes
Step 136500 of 400000; TPS: 2903.03; ETA: 1.5 minutes
Step 147000 of 400000; TPS: 2904.8; ETA: 1.5 minutes
Step 157500 of 400000; TPS: 2904.91; ETA: 1.4 minutes
Step 168000 of 400000; TPS: 2904.28; ETA: 1.3 minutes
Step 178500 of 400000; TPS: 2903.31; ETA: 1.3 minutes
Step 189000 of 400000; TPS: 2901.66; ETA: 1.2 minutes
Step 199500 of 400000; TPS: 2898.95; ETA

In [ ]:
cg_sim.box_lengths

In [ ]:
target_box